In [1]:
import numpy as np
import pandas as pd
import pickle
import csv
import keras as K
from os import listdir
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dense
from keras.applications import densenet
from IPython.display import SVG
from keras.optimizers import Adam
from PIL import Image
import scipy
from skimage.transform import resize
from matplotlib.pyplot import imshow
from keras.preprocessing.text import Tokenizer
from keras.applications.imagenet_utils import preprocess_input

from keras.models import load_model

Using TensorFlow backend.


In [2]:
input_shape = (224,224,3)
class_num = 14

In [3]:
weights_path = 'brucechou1983_CheXNet_weights.h5'

In [4]:
img_input = Input(shape=input_shape)

In [5]:
base_model = densenet.DenseNet121(include_top=False, weights=None, input_tensor=img_input, input_shape=input_shape, pooling="avg")

In [6]:
X = base_model.output
predictions = Dense(class_num, activation="sigmoid", name="predictions")(X)
model = Model(inputs=img_input, outputs=predictions)

In [7]:
model.load_weights(weights_path)

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

__________________________________________________________________________________________________
conv5_block10_concat (Concatena (None, 7, 7, 832)    0           conv5_block9_concat[0][0]        
                                                                 conv5_block10_2_conv[0][0]       
__________________________________________________________________________________________________
conv5_block11_0_bn (BatchNormal (None, 7, 7, 832)    3328        conv5_block10_concat[0][0]       
__________________________________________________________________________________________________
conv5_block11_0_relu (Activatio (None, 7, 7, 832)    0           conv5_block11_0_bn[0][0]         
__________________________________________________________________________________________________
conv5_block11_1_conv (Conv2D)   (None, 7, 7, 128)    106496      conv5_block11_0_relu[0][0]       
__________________________________________________________________________________________________
conv5_bloc

In [9]:
layer_name = 'avg_pool'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)

In [10]:
visual_feature = {}

In [11]:
dataset_dir = "F:\\Projects\\MajorProject\\Dataset\\NLMCXR_png"

In [13]:
def load_img_arr(img_name):
    image = Image.open(dataset_dir + "\\" + img_name + ".png")
    image_array = np.asarray(image.convert("RGB"))
    image_array = image_array / 255.
    image_array = resize(image_array, (224,224))
    X = np.expand_dims(image_array, axis=0)
    X = np.asarray(X)
    
    return X

In [14]:
with open("preprocess.csv") as f:
    readCSV = csv.reader(f)
    i=0
    for row in readCSV:
        if i>0:
            print("file_Name : {0} , image1: {1} , image2: {2}".format(row[0], row[1],row[2]))
            
            X_first = load_img_arr(row[1])
            intermediate_output_1 = intermediate_layer_model.predict(X_first)
            
            X_second = load_img_arr(row[2])
            intermediate_output_2 = intermediate_layer_model.predict(X_second)
            
            final_output = np.concatenate((intermediate_output_1, intermediate_output_2), axis=1)
            print(final_output.shape)
            visual_feature[row[0]] = final_output
        i = i+1

file_Name : 1.xml , image1: CXR1_1_IM-0001-3001 , image2: CXR1_1_IM-0001-4001


C:\Users\Aniket\Anaconda2\envs\py36\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\Aniket\Anaconda2\envs\py36\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


(1, 2048)
file_Name : 10.xml , image1: CXR10_IM-0002-1001 , image2: CXR10_IM-0002-2001
(1, 2048)
file_Name : 100.xml , image1: CXR100_IM-0002-1001 , image2: CXR100_IM-0002-2001
(1, 2048)
file_Name : 1001.xml , image1: CXR1001_IM-0004-1001 , image2: CXR1001_IM-0004-1002
(1, 2048)
file_Name : 1002.xml , image1: CXR1002_IM-0004-1001 , image2: CXR1002_IM-0004-2001
(1, 2048)
file_Name : 1004.xml , image1: CXR1004_IM-0005-1001 , image2: CXR1004_IM-0005-2001
(1, 2048)
file_Name : 1005.xml , image1: CXR1005_IM-0006-1001 , image2: CXR1005_IM-0006-3003
(1, 2048)
file_Name : 1006.xml , image1: CXR1006_IM-0007-1001 , image2: CXR1006_IM-0007-3003
(1, 2048)
file_Name : 1008.xml , image1: CXR1008_IM-0009-2001 , image2: CXR1008_IM-0009-4004
(1, 2048)
file_Name : 1009.xml , image1: CXR1009_IM-0010-1001 , image2: CXR1009_IM-0010-2001
(1, 2048)
file_Name : 101.xml , image1: CXR101_IM-0011-2001 , image2: CXR101_IM-0011-4004
(1, 2048)
file_Name : 1010.xml , image1: CXR1010_IM-0012-1001 , image2: CXR1010_IM

(1, 2048)
file_Name : 1103.xml , image1: CXR1103_IM-0070-1001 , image2: CXR1103_IM-0070-2001
(1, 2048)
file_Name : 1105.xml , image1: CXR1105_IM-0072-1001-0001 , image2: CXR1105_IM-0072-1001-0002
(1, 2048)
file_Name : 1106.xml , image1: CXR1106_IM-0073-1001 , image2: CXR1106_IM-0073-2001
(1, 2048)
file_Name : 1108.xml , image1: CXR1108_IM-0075-1001 , image2: CXR1108_IM-0075-2001
(1, 2048)
file_Name : 1109.xml , image1: CXR1109_IM-0076-1001 , image2: CXR1109_IM-0076-2001
(1, 2048)
file_Name : 111.xml , image1: CXR111_IM-0076-1001 , image2: CXR111_IM-0076-1002
(1, 2048)
file_Name : 1110.xml , image1: CXR1110_IM-0076-1001 , image2: CXR1110_IM-0076-2001
(1, 2048)
file_Name : 1111.xml , image1: CXR1111_IM-0077-1001 , image2: CXR1111_IM-0077-4004
(1, 2048)
file_Name : 1112.xml , image1: CXR1112_IM-0078-1001 , image2: CXR1112_IM-0078-2001
(1, 2048)
file_Name : 1113.xml , image1: CXR1113_IM-0078-1001 , image2: CXR1113_IM-0078-1002
(1, 2048)
file_Name : 1114.xml , image1: CXR1114_IM-0079-1001 ,

(1, 2048)
file_Name : 1205.xml , image1: CXR1205_IM-0138-1001 , image2: CXR1205_IM-0138-1002
(1, 2048)
file_Name : 1206.xml , image1: CXR1206_IM-0139-1001 , image2: CXR1206_IM-0139-3003
(1, 2048)
file_Name : 1207.xml , image1: CXR1207_IM-0140-0001-0001 , image2: CXR1207_IM-0140-0001-0002
(1, 2048)
file_Name : 1209.xml , image1: CXR1209_IM-0142-1001 , image2: CXR1209_IM-0142-1002
(1, 2048)
file_Name : 121.xml , image1: CXR121_IM-0142-1001 , image2: CXR121_IM-0142-2001
(1, 2048)
file_Name : 1210.xml , image1: CXR1210_IM-0142-1001 , image2: CXR1210_IM-0142-2001
(1, 2048)
file_Name : 1211.xml , image1: CXR1211_IM-0142-1001 , image2: CXR1211_IM-0142-1002
(1, 2048)
file_Name : 1212.xml , image1: CXR1212_IM-0143-1001 , image2: CXR1212_IM-0143-2001
(1, 2048)
file_Name : 1213.xml , image1: CXR1213_IM-0144-1001 , image2: CXR1213_IM-0144-2001
(1, 2048)
file_Name : 1214.xml , image1: CXR1214_IM-0144-1001 , image2: CXR1214_IM-0144-2001
(1, 2048)
file_Name : 1217.xml , image1: CXR1217_IM-0145-0001-0

...

(1, 2048)
file_Name : 578.xml , image1: CXR578_IM-2176-1001 , image2: CXR578_IM-2176-2001
(1, 2048)
file_Name : 579.xml , image1: CXR579_IM-2176-1001 , image2: CXR579_IM-2176-1002
(1, 2048)
file_Name : 58.xml , image1: CXR58_IM-2177-1001 , image2: CXR58_IM-2177-2001
(1, 2048)
file_Name : 580.xml , image1: CXR580_IM-2177-2002 , image2: CXR580_IM-2177-3003
(1, 2048)
file_Name : 581.xml , image1: CXR581_IM-2178-1001 , image2: CXR581_IM-2178-3001
(1, 2048)
file_Name : 582.xml , image1: CXR582_IM-2179-1001 , image2: CXR582_IM-2179-2001
(1, 2048)
file_Name : 583.xml , image1: CXR583_IM-2180-1001 , image2: CXR583_IM-2180-2001
(1, 2048)
file_Name : 584.xml , image1: CXR584_IM-2181-1001 , image2: CXR584_IM-2181-2001
(1, 2048)
file_Name : 585.xml , image1: CXR585_IM-2181-1001 , image2: CXR585_IM-2181-2001
(1, 2048)
file_Name : 588.xml , image1: CXR588_IM-2183-1001 , image2: CXR588_IM-2183-2001
(1, 2048)
file_Name : 59.xml , image1: CXR59_IM-2184-1001 , image2: CXR59_IM-2184-2001
(1, 2048)
file_N

(1, 2048)
file_Name : 675.xml , image1: CXR675_IM-2247-1001 , image2: CXR675_IM-2247-2001
(1, 2048)
file_Name : 676.xml , image1: CXR676_IM-2248-1001 , image2: CXR676_IM-2248-2001
(1, 2048)
file_Name : 677.xml , image1: CXR677_IM-2249-1001 , image2: CXR677_IM-2249-2001
(1, 2048)
file_Name : 679.xml , image1: CXR679_IM-2251-1001 , image2: CXR679_IM-2251-2001
(1, 2048)
file_Name : 680.xml , image1: CXR680_IM-2251-1001 , image2: CXR680_IM-2251-2001
(1, 2048)
file_Name : 681.xml , image1: CXR681_IM-2252-0001-0001 , image2: CXR681_IM-2252-0001-0002
(1, 2048)
file_Name : 682.xml , image1: CXR682_IM-2253-1001 , image2: CXR682_IM-2253-2001
(1, 2048)
file_Name : 685.xml , image1: CXR685_IM-2254-1001 , image2: CXR685_IM-2254-2001
(1, 2048)
file_Name : 686.xml , image1: CXR686_IM-2254-1001 , image2: CXR686_IM-2254-2001
(1, 2048)
file_Name : 687.xml , image1: CXR687_IM-2255-1001 , image2: CXR687_IM-2255-2001
(1, 2048)
file_Name : 688.xml , image1: CXR688_IM-2256-1001 , image2: CXR688_IM-2256-2001


(1, 2048)
file_Name : 783.xml , image1: CXR783_IM-2325-1001 , image2: CXR783_IM-2325-2001
(1, 2048)
file_Name : 784.xml , image1: CXR784_IM-2325-1001-0001 , image2: CXR784_IM-2325-1001-0002
(1, 2048)
file_Name : 785.xml , image1: CXR785_IM-2325-1001 , image2: CXR785_IM-2325-2001
(1, 2048)
file_Name : 788.xml , image1: CXR788_IM-2328-1001 , image2: CXR788_IM-2328-2001
(1, 2048)
file_Name : 79.xml , image1: CXR79_IM-2329-1001 , image2: CXR79_IM-2329-2001
(1, 2048)
file_Name : 790.xml , image1: CXR790_IM-2329-1001 , image2: CXR790_IM-2329-1002
(1, 2048)
file_Name : 791.xml , image1: CXR791_IM-2330-1001 , image2: CXR791_IM-2330-2001
(1, 2048)
file_Name : 792.xml , image1: CXR792_IM-2330-1001 , image2: CXR792_IM-2330-2002
(1, 2048)
file_Name : 793.xml , image1: CXR793_IM-2330-1001 , image2: CXR793_IM-2330-2001
(1, 2048)
file_Name : 794.xml , image1: CXR794_IM-2331-1001 , image2: CXR794_IM-2331-2001
(1, 2048)
file_Name : 795.xml , image1: CXR795_IM-2331-1001 , image2: CXR795_IM-2331-1002
(1,

(1, 2048)
file_Name : 889.xml , image1: CXR889_IM-2401-1001 , image2: CXR889_IM-2401-2001
(1, 2048)
file_Name : 89.xml , image1: CXR89_IM-2402-1001 , image2: CXR89_IM-2402-2001
(1, 2048)
file_Name : 890.xml , image1: CXR890_IM-2403-1001 , image2: CXR890_IM-2403-2001
(1, 2048)
file_Name : 891.xml , image1: CXR891_IM-2403-1001 , image2: CXR891_IM-2403-2001
(1, 2048)
file_Name : 892.xml , image1: CXR892_IM-2403-1001 , image2: CXR892_IM-2403-2001
(1, 2048)
file_Name : 893.xml , image1: CXR893_IM-2403-1001 , image2: CXR893_IM-2403-2001
(1, 2048)
file_Name : 894.xml , image1: CXR894_IM-2404-0001-0001 , image2: CXR894_IM-2404-0001-0002
(1, 2048)
file_Name : 897.xml , image1: CXR897_IM-2406-1001 , image2: CXR897_IM-2406-2001
(1, 2048)
file_Name : 899.xml , image1: CXR899_IM-2407-1001 , image2: CXR899_IM-2407-2001
(1, 2048)
file_Name : 9.xml , image1: CXR9_IM-2407-1001 , image2: CXR9_IM-2407-2001
(1, 2048)
file_Name : 90.xml , image1: CXR90_IM-2407-1001 , image2: CXR90_IM-2407-2001
(1, 2048)
fi

(1, 2048)
file_Name : 99.xml , image1: CXR99_IM-2476-1001 , image2: CXR99_IM-2476-1002
(1, 2048)
file_Name : 990.xml , image1: CXR990_IM-2476-1001 , image2: CXR990_IM-2476-2001
(1, 2048)
file_Name : 991.xml , image1: CXR991_IM-2476-1001 , image2: CXR991_IM-2476-2001
(1, 2048)
file_Name : 992.xml , image1: CXR992_IM-2477-0001-0001 , image2: CXR992_IM-2477-0001-0002
(1, 2048)
file_Name : 993.xml , image1: CXR993_IM-2478-1001 , image2: CXR993_IM-2478-1002
(1, 2048)
file_Name : 994.xml , image1: CXR994_IM-2478-1001 , image2: CXR994_IM-2478-2001
(1, 2048)
file_Name : 995.xml , image1: CXR995_IM-2478-1001 , image2: CXR995_IM-2478-1002
(1, 2048)
file_Name : 996.xml , image1: CXR996_IM-2479-1001 , image2: CXR996_IM-2479-2001
(1, 2048)
file_Name : 997.xml , image1: CXR997_IM-2479-1001 , image2: CXR997_IM-2479-2001
(1, 2048)
file_Name : 998.xml , image1: CXR998_IM-2479-1001 , image2: CXR998_IM-2479-2001
(1, 2048)


In [15]:
with open('visual_features.pickle', 'wb') as handle:
    pickle.dump(visual_feature, handle, protocol=pickle.HIGHEST_PROTOCOL)